<a href="https://colab.research.google.com/github/holgerfach/dw_matrix/blob/master/day3_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [9]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


#Wczytywanie danych

In [10]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [11]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

#Dummy Model

In [12]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [18]:
feats = ['car_id']
x = df[ feats ].values
y = df ['price_value'].values

model = DummyRegressor()
model.fit(x, y)
y_pred = model.predict(x)

mae(y, y_pred)

39465.934630440985

In [19]:
[x for x in df.columns if 'price' in x] 

['price_currency', 'price_details', 'price_value']

In [22]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [24]:
df = df[ df['price_currency'] != 'EUR' ]
df.shape

(106290, 155)

##Features

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [33]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [35]:
x = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [37]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(x, y)


imp = PermutationImportance(m).fit(x,y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2578 ± 0.0070,param_napęd__cat
0.2016 ± 0.0058,param_faktura-vat__cat
0.1930 ± 0.0063,param_stan__cat
0.1436 ± 0.0050,param_rok-produkcji__cat
0.0638 ± 0.0042,param_moc__cat
0.0418 ± 0.0008,feature_kamera-cofania__cat
0.0406 ± 0.0031,param_skrzynia-biegów__cat
0.0273 ± 0.0042,param_marka-pojazdu__cat
0.0190 ± 0.0012,param_pojemność-skokowa__cat
0.0164 ± 0.0004,feature_bluetooth__cat
